In [2]:

from utils.config_loader import app_config
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext
from llama_index.vector_stores.milvus import MilvusVectorStore

from pymilvus import connections, list_collections
import textwrap

from llama_index.core.vector_stores import MetadataFilters,  ExactMatchFilter


In [3]:
import logging
logging.basicConfig(level=logging.ERROR)

### Example 1: Using LlamaIndex to query the Milvus vector store

In [4]:
# Step 0 . Connect to Milvus
try:
    # 1. Connect to Milvus
    milvus_store = MilvusVectorStore(
        uri=app_config.MILVUS_URL,
        dim=app_config.MILVUS_DIM,        
        overwrite=False,
        collection_name="_emails_attachments"
    )
    print("Connected to Milvus successfully.")
except Exception as e:
    print(f"Failed to connect to Milvus: {e}")

2025-03-27 12:19:16,197 [DEBUG][_create_connection]: Created new connection using: d549c43e311f4b1f8c68ffa194a62167 (async_milvus_client.py:600)


Connected to Milvus successfully.


In [5]:
# Step 0: Optional. To see the contents of a search result. 

print(milvus_store.client.search(
    collection_name="_emails_attachments", 
    data=[[0.1] * app_config.MILVUS_DIM],
    limit=1,
    output_fields=["*"],
))

data: ['[{\'id\': \'8a29b96c-8a53-414f-98bf-5e5de5bb3d55\', \'distance\': -0.11188199371099472, \'entity\': {\'id\': \'8a29b96c-8a53-414f-98bf-5e5de5bb3d55\', \'embedding\': [-0.015253296121954918, -0.009743201546370983, -0.01674899272620678, -0.0027338815852999687, -0.009912525303661823, 0.028898010030388832, -0.0057817185297608376, -0.021151425316929817, -0.025356311351060867, -0.013150852173566818, 0.020601121708750725, 0.0009004171588458121, -0.01952873356640339, -0.014590108767151833, -0.015930593013763428, 0.007817137986421585, 0.0016500299097970128, -0.019810941070318222, -0.01471710205078125, -0.009072959423065186, 0.004536479711532593, 0.02031891420483589, -0.01237478293478489, -0.007859468460083008, 0.017002979293465614, -0.0001754975673975423, 0.026132380589842796, -0.036715149879455566, -0.01173276174813509, -0.004282493144273758, 0.00022345072648022324, 0.007753640878945589, -0.01885143667459488, -0.011344727128744125, -0.027529306709766388, -0.030280826613307, -0.00924228

In [6]:
## Step 1: Prepare global settings

# Access appplication environment variables
model = app_config.LLAMAINDEX_MODEL
embed_model= app_config.LLAMAINDEX_EMBEDDING

# define global Setting that sets the LLM and ebemdding model.
# There is no need to pass in any llm or embedding model to the LlamaIndexTool query engine.
# The LlamaIndexTool will use the global Settings to get the llm and embedding model.
Settings.llm = OpenAI(temperature=0.1, model=model)
Settings.embed_model = OpenAIEmbedding(model=embed_model)

In [7]:
# Step 2: Load index from existing Milvus vector store
index = VectorStoreIndex.from_vector_store(
    vector_store=milvus_store
)

In [16]:
# Step 3. Create the query and chat engine using the LLM
query_engine = index.as_query_engine()  # no parameter needed, taken care off by the global Settings
chat_engine = index.as_chat_engine()


queries = [
    "what did Juliet Iris Lee reqeusted for?",
    "what are items shipped?",
    "where is the destination terminal of the shipment?  Provide the destination address.",
    "find items names and qauntity in move order 6009981",
    "when is the requested truck in date and time?",
    "identify the vessel name and voyage number for the job",
    ## add more queries here ##
]

In [17]:

# Step 4A. Run your query
for q in queries:
    response = query_engine.query(q)
    print(textwrap.fill(response.response, width=80))
    print()

The information provided does not specify any particular request made by Juliet
Iris Lee.

The items shipped are various types of wheat flour, including CLOVER (KF) WHEAT
FLOUR, IVORY (F) WHEAT FLOUR, TOWER (F) WHEAT FLOUR, and FAIRY (F) WHEAT FLOUR.

The destination terminal of the shipment is Jakarta. However, the specific
address of the destination terminal is not provided.

The items and their quantities in move order 6009981 are as follows:  1. CLOVER
(KF) WHEAT FLOUR @ 25 KG NETT - 72 bags 2. IVORY (F) WHEAT FLOUR @ 25 KG NETT -
72 bags 3. TOWER (F) WHEAT FLOUR @ 20 KG - 80 bags 4. FAIRY (F) WHEAT FLOUR @ 25
KG NETT - 646 bags

The requested truck-in date is on 02 October, which falls on a Wednesday. There
is no specific time mentioned.

The vessel name and voyage number are not provided in the available information.



In [15]:
# Step 4B. Run your  chat engine

for q in queries:
    response = chat_engine.chat(q)
    print(textwrap.fill(response.response, width=80))
    print()



There is no specific information available about any requests made by Juliet
Iris Lee.

The items shipped include:  - CLOVER (KF) WHEAT FLOUR - IVORY (F) WHEAT FLOUR -
TOWER (F) WHEAT FLOUR - FAIRY (F) WHEAT FLOUR

The destination terminal and address for the shipment are not specified in the
available information.

In move order 6009981, the items and their quantities are:  1. CLOVER (KF) WHEAT
FLOUR @ 25 KG NETT - 72 bags 2. IVORY (F) WHEAT FLOUR @ 25 KG NETT - 72 bags 3.
TOWER (F) WHEAT FLOUR @ 20 KG - 80 bags 4. FAIRY (F) WHEAT FLOUR @ 25 KG NETT -
646 bags

The requested truck-in date for move order 6009981 is October 2nd. However,
there is no specific time mentioned.

The vessel name and voyage number for move order 6009981 are not specified in
the available information.



### Example 2: Using LlamaIndex to query the Milvus vector store using Meta Data Filters

In [11]:

query_engine_with_filters = index.as_query_engine(
        include_metadata=True,
        vector_store_query_mode="default",
        filters=MetadataFilters(
        filters=[
           # ExactMatchFilter(key="threadID", value="194d494969f09f62"),
        ]
        )
)

for q in queries:
    response = query_engine_with_filters.query(q)
    print(textwrap.fill(response.response, width=80))
    print()

The context does not provide specific details about any request made by Juliet
Iris Lee.

The items shipped are various types of wheat flour, including CLOVER (KF) WHEAT
FLOUR, IVORY (F) WHEAT FLOUR, TOWER (F) WHEAT FLOUR, and FAIRY (F) WHEAT FLOUR.

The destination terminal of the shipment is Jakarta. However, the specific
address for the destination terminal is not provided.

The items and their quantities in move order 6009981 are as follows:  1. CLOVER
(KF) WHEAT FLOUR @ 25 KG NETT - 72 bags 2. IVORY (F) WHEAT FLOUR @ 25 KG NETT -
72 bags 3. TOWER (F) WHEAT FLOUR @ 20 KG - 80 bags 4. FAIRY (F) WHEAT FLOUR @ 25
KG NETT - 646 bags

The requested truck-in date is 02 October, and no specific time is mentioned.

